<a href="https://colab.research.google.com/github/ARULDHAKSHANAMOORTHI/Projects/blob/main/Chatbot_using_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Libraries needed for NLP
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
stemmer=PorterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#libraries needed for tensor flow
import tensorflow as tf
import numpy as np
import random
import json

In [ ]:
#load intents.json file from computer
from google.colab import files
files.upload()
#import our chatbot intent files
with open('intents.json') as json_data:
  intents=json.load(json_data)

Saving intents.json to intents (1).json


In [ ]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'chatbot',
   'patterns': ['Who built this chatbot?',
    'Tell me about Chatbot',
    'What is this chatbot name?'],
   'responses': ['Hi, I am Chatbot designed by Arul.',
    'Thanks for asking. I am designed by Arul.',
    'I am a chatbot.']},
  {'tag': 'location',
   'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?'],
   'responses': ["We are from World's larges

In [ ]:
#Data preprocessing
words=[]
classes=[]
documents=[]
ignore=['?']

**Tokenizing**

In [ ]:
#loop through each sentences in the intent's pattern
for intent in intents['intents']:
  for pattern in intent['patterns']:
    #Tokenize each word by removing the suffix or prefixes
    w=nltk.word_tokenize(pattern)
    #add word to the words list
    words.extend(w)
    #add words to documents
    documents.append((w,intent['tag']))
    #add tags to our classes list
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

**perform stemming and lower each words as well as removing duplicates**

In [ ]:
words=[stemmer.stem(w.lower()) for w in words if w not in ignore ]
words=sorted(list(set(words)))
print(words)
print(len(words),'words')
print(len(documents),'documents')
print(len(classes),'classes')

["'s", 'about', 'account', 'address', 'ani', 'anyon', 'are', 'built', 'bye', 'can', 'chatbot', 'connect', 'day', 'favourit', 'give', 'good', 'goodbi', 'goodby', 'hello', 'help', 'hi', 'how', 'i', 'is', 'later', 'link', 'locat', 'me', 'media', 'movi', 'name', 'out', 'reach', 'recommend', 'see', 'social', 'some', 'suggest', 'tell', 'thank', 'that', 'there', 'thi', 'to', 'way', 'we', 'what', 'where', 'which', 'who', 'you', 'your', 'yourself']
53 words
54 documents
8 classes


**creating and training the model**

In [ ]:
training=[]  #traing data as x
output=[] #testing data as y

In [ ]:
output_empty=[0]*len(classes)

In [ ]:
# creating training set ,bag of words for each sentence
for doc in documents:
  #intialize bag of words
  bag=[]
  #list of tokenized words for the pattern
  pattern_words=doc[0]
  #stemming each words
  pattern_words=[stemmer.stem(word.lower()) for word in pattern_words]       #doubt...?
  #Create bag of words array
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)
  output_row=list(output_empty)
  output_row[classes.index(doc[1])]
  training.append([bag,output_row])
# shuffling features and turning it into np.array
random.shuffle(training)
training = np.array(training,dtype='object')

# creating training lists
train_x = list(training[:,0])
train_y = list(training[:,1])


In [ ]:
model=tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10,input_shape=(len(train_x[0]),)))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Dense(len(train_y[0]),activation="softmax"))
model.compile(tf.keras.optimizers.Adam(),loss='categorical_crossentropy',metrics=['accuracy'])
#train a model
model.fit(np.array(train_x),np.array(train_y),epochs=100,batch_size=8,verbose=1)
model.save('model.pkl')

Epoch 1/100
34/34 [==============================] - 1s 4ms/step - loss: 0.0000e+00 - accuracy: 0.0741
Epoch 2/100
34/34 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 0.0741
Epoch 3/100
34/34 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 0.0741
Epoch 4/100
34/34 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 0.0741
Epoch 5/100
34/34 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 0.0889
Epoch 6/100
34/34 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 0.1111
Epoch 7/100
34/34 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0667
Epoch 8/100
34/34 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 0.0407
Epoch 9/100
34/34 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 0.0296
Epoch 10/100
34/34 [==============================] - 0s 3ms/step - loss:

**Making predictions using ChatBot**

In [ ]:
import pickle
pickle.dump({'words':words,'classes':classes},open('training_data','wb'))
from keras.models import load_model
model=load_model('model.pkl')

In [ ]:
#restoring all the data Structures
data=pickle.load(open('training_data','rb'))
words=data['words']
classes=data['classes']
with open ('intents.json') as json_data:
  intents=json.load(json_data)

In [ ]:
def clean_up_sentence(sentence):
  #tokenizing the pattern
  sentence_words=nltk.word_tokenize(sentence)
  #Stemming each word
  sentence_words=[stemmer.stem(word.lower()) for word in sentence_words]
  return sentence_words
#returning bag of words array :0 or 1 for each word in bag that exists in the sentence
def bow(sentence,words):
  #tokenizing the pattern
  sentence_words=clean_up_sentence(sentence)
  #generating bag of words
  bag=[0]*len(words)
  for s in sentence_words:
    for i,w in enumerate(words):
      if w==s:
        bag[i]=1
  bag=np.array(bag)
  return bag

In [ ]:
ERROR_THRESHOLD =0.30
def classify(sentence):
  #generate probabilities from the model
  bag=bow(sentence,words)
  results=model.predict(np.array([bag]))
  #filter out predictions below threshold
  results=[[i,r] for i,r in enumerate(results[0]) if r>ERROR_THRESHOLD]
  #sort by strength of probibility
  results.sort(key=lambda x:x[1],reverse=True)
  return_list=[]
  for r in results:
    return_list.append((classes[r[0]],r[1]))
  #return tuple of intent and probability
  return return_list

In [ ]:
def response(sentence):
  results=classify(sentence)
  #if we have a classification then find matching intent
  if results:
    #loop as long as there as matches to process
    while results:
      for i in intents['intents']:
        #find a tag matching the first results
        if i['tag']==results[0][0]:
          #random response from the intent
          return print(random.choice(i['responses']))
      results.pop(0)


In [ ]:
response('Where is your located?')

1/1 [==============================] - 0s 25ms/step
I am a chatbot.
